In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install tfa-nightly
!pip install import-ipynb
import import_ipynb
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
import os
import scipy.io as sio
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from tqdm import tqdm
from fastprogress import master_bar, progress_bar
%cd '/content/drive/MyDrive/LMSLoss'

     |████████████████████████████████| 747kB 6.0MB/s 
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=9453fcfd948894ddbf697f780feeca74d79b0e677de6e30cadd7f47e9445b607
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
/content/drive/MyDrive/LMSLoss


In [3]:
%run UnetModel.ipynb
# %run MumfordLoss.ipynb
%run augment_data.ipynb


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 576         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
swish (Swish)                   (None, 128, 128, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [4]:
import keras.backend as K

def dice_coef(y_true, y_pred, smooth=1e-10):
    '''Average dice coefficient per batch.'''
    axes = (1,2,3)
    intersection = K.sum(y_true * y_pred, axis=axes)
    summation = K.sum(y_true + y_pred, axis=axes)
    return K.mean((2.0 * intersection + smooth) / (summation + smooth), axis=0)

def jaccard_coef(y_true, y_pred, smooth=1e-10):
    '''Average jaccard coefficient per batch.'''
    axes = (1,2,3)
    intersection = K.sum(y_true * y_pred, axis=axes)
    union = K.sum(y_true + y_pred, axis=axes) - intersection
    return K.mean( (intersection + smooth) / (union + smooth), axis=0)
def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred, 0.0)

In [5]:
x =  np.load("dataISIC2018/x.npy")
y = np.load("dataISIC2018/y.npy")
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state =42)
x_train=tf.convert_to_tensor(x_train,tf.float32)
y_train=tf.convert_to_tensor(y_train,tf.float32)
x_test=tf.convert_to_tensor(x_test,tf.float32)
y_test=tf.convert_to_tensor(y_test,tf.float32)


In [6]:
BATCH_SIZE = 32
buffer_size = x_train.shape[0]

@tf.function
def gen_image(image,mask):
  # image, mask = brightness(image,mask)
  #image, mask = saturation(image,mask)
  image, mask = flip(image,mask)
  image, mask = rotate_image(image,mask)
  image, mask = normalize(image, mask)
  return image, mask
train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(buffer_size).map(gen_image).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test)).map(normalize).batch(BATCH_SIZE*2)
# val_dataset = tf.data.Dataset.from_tensor_slices((x_val,y_val)).map(normalize).batch(BATCH_SIZE*2)

In [9]:
S = seg_net((128,160,3),out_channels=1)
S.compile(loss = dice_coef_loss,optimizer=tf.keras.optimizers.Nadam(0.001),metrics=[dice_coef, jaccard_coef])

In [10]:
mcp = ModelCheckpoint("./weightISIC_diceloss/weight{val_dice_coef:.4f}.h5",mode='max',monitor='val_dice_coef',verbose=1,save_best_only=True,save_weights_only=True)
earlystop= EarlyStopping(monitor='val_dice_coef',patience=5,mode='max',verbose=1)
rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, mode='max', patience=10, min_lr=1e-6, verbose=1)
history = S.fit(train_dataset,validation_data=test_dataset,
	    epochs=300,callbacks=[earlystop,mcp],verbose=1)

Epoch 1/300
65/65 [==============================] - ETA: 0s - loss: 0.3985 - dice_coef: 0.6018 - jaccard_coef: 0.4875WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0114s vs `on_test_batch_end` time: 0.2941s). Check your callbacks.

Epoch 00001: val_dice_coef improved from -inf to 0.32995, saving model to ./weightISIC_diceloss/weight0.3300.h5
65/65 [==============================] - 62s 949ms/step - loss: 0.3985 - dice_coef: 0.6018 - jaccard_coef: 0.4875 - val_loss: 0.6931 - val_dice_coef: 0.3300 - val_jaccard_coef: 0.2335
Epoch 2/300
65/65 [==============================] - ETA: 0s - loss: 0.2203 - dice_coef: 0.7795 - jaccard_coef: 0.6815
Epoch 00002: val_dice_coef improved from 0.32995 to 0.48446, saving model to ./weightISIC_diceloss/weight0.4845.h5
65/65 [==============================] - 61s 937ms/step - loss: 0.2203 - dice_coef: 0.7795 - jaccard_coef: 0.6815 - val_loss: 0.5309 - val_dice_coef: 0.4845 - val_jaccard_coef: 0

In [11]:
sio.savemat('./weightISIC_diceloss/historyISIC_DL.mat',history.history)